In [1]:
from pymongo import MongoClient
from lxml import html

import os

mongo_connection_uri = os.environ.get("ME_CONFIG_MONGODB_URL")
client = MongoClient(mongo_connection_uri)

db = client["atcoder"]
contest_raw_collection = db["contest_raw"]
contest_collection = db["contest"]

In [2]:
contest_list = []

for contest_raw in contest_raw_collection.find():
    tree = html.fromstring(contest_raw["raw_html"])
    tr_list = tree.xpath("//tbody/tr")
    for tr_elem in tr_list:
        start_time = tr_elem.xpath(".//time/text()")[0]
        contest_elem = tr_elem.xpath(".//td[2]//a")[0]
        contest_collection.insert_one(
            {
                "start_time": start_time,
                "contest_url": contest_elem.get("href"),
                "contest_name": contest_elem.text_content(),
            }
        )